<a href="https://colab.research.google.com/github/Computer-CGuy/BitcoinTPU/blob/main/OldSongs_%7C_NewEdition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import jax.tools.colab_tpu
jax.tools.colab_tpu.setup_tpu()

In [2]:
import jax

In [3]:
import numpy as np

In [4]:
sha_k =  np.array([ 0x428A2F98, 0x71374491, 0xB5C0FBCF, 0xE9B5DBA5, 0x3956C25B, 0x59F111F1, 0x923F82A4, 0xAB1C5ED5,\
 0xD807AA98, 0x12835B01, 0x243185BE, 0x550C7DC3, 0x72BE5D74, 0x80DEB1FE, 0x9BDC06A7, 0xC19BF174,\
 0xE49B69C1, 0xEFBE4786, 0x0FC19DC6, 0x240CA1CC, 0x2DE92C6F, 0x4A7484AA, 0x5CB0A9DC, 0x76F988DA,\
 0x983E5152, 0xA831C66D, 0xB00327C8, 0xBF597FC7, 0xC6E00BF3, 0xD5A79147, 0x06CA6351, 0x14292967,\
 0x27B70A85, 0x2E1B2138, 0x4D2C6DFC, 0x53380D13, 0x650A7354, 0x766A0ABB, 0x81C2C92E, 0x92722C85,\
 0xA2BFE8A1, 0xA81A664B, 0xC24B8B70, 0xC76C51A3, 0xD192E819, 0xD6990624, 0xF40E3585, 0x106AA070,\
 0x19A4C116, 0x1E376C08, 0x2748774C, 0x34B0BCB5, 0x391C0CB3, 0x4ED8AA4A, 0x5B9CCA4F, 0x682E6FF3,\
 0x748F82EE, 0x78A5636F, 0x84C87814, 0x8CC70208, 0x90BEFFFA, 0xA4506CEB, 0xBEF9A3F7, 0xC67178F2],dtype=np.uint32)

In [5]:
listed_w = np.random.randint(low=0,high=999,size=(16,),dtype=int)#[0xD807AA98 for x in range(64-16)]

In [6]:
class WhoMe():
    def __init__(self,w):
        self.w = w
    def __getitem__(self,indices):
        # print(type(indices))
        return getW(indices,self.w)

In [7]:
def getW(i,w):
    if(i<16):
        return listed_w[i]
    else:
        return w[i-16]

In [ ]:
def roundtill16(a,b,c,d,e,f,g,h,w_i,sha_k_i):
    S0 = (a >> 2 | a << (32-2)) ^ (a >> 13 | a << (32-13)) ^ (a >> 22 | a << (32-22));
    maj = (a & b) ^ (a & c) ^ (b & c);
    t2 = S0 + maj;
    S1 = (e >> 6 | e << (32-6)) ^ (e >> 11 | e << (32-11)) ^ (e >> 25 | e << (32-25));
    ch = (e & f) ^ ((~e) & g);
    t1 = h + S1 + ch + sha_k_i + w_i;#sha_k[n] + w[n];

    h = g; g = f; f = e; e = d + t1;
    d = c; c = b; b = a; a = t1 + t2;
    return a,b,c,d,e,f,g,h

In [8]:
def func(w_):
    w =WhoMe(w_)
    print("hello")
    a,b,c,d,e,f,g,h = [0x6A09E667, 0xBB67AE85, 0x3C6EF372, 0xA54FF53A, 0x510E527F, 0x9B05688C, 0x1F83D9AB, 0x5BE0CD19]
    for n in range(16):
        S0 = (a >> 2 | a << (32-2)) ^ (a >> 13 | a << (32-13)) ^ (a >> 22 | a << (32-22));
        maj = (a & b) ^ (a & c) ^ (b & c);
        t2 = S0 + maj;
        S1 = (e >> 6 | e << (32-6)) ^ (e >> 11 | e << (32-11)) ^ (e >> 25 | e << (32-25));
        ch = (e & f) ^ ((~e) & g);
        t1 = h + S1 + ch + sha_k[n] + w[n];

        h = g; g = f; f = e; e = d + t1;
        d = c; c = b; b = a; a = t1 + t2;
    for n in range(16,64):
        s0 = (w[n-15] >> 7 | w[n-15] << (32-7)) ^ (w[n-15] >> 18 | w[n-15] << (32-18)) ^ (w[n-15] >> 3);
        s1 = (w[n-2] >> 17 | w[n-2] << (32-17)) ^ (w[n-2] >> 19 | w[n-2] << (32-19)) ^ (w[n-2] >> 10);
        new_var = w[n-16] + s0 + w[n-7] + s1;
        
        S0 = (a >> 2 | a << (32-2)) ^ (a >> 13 | a << (32-13)) ^ (a >> 22 | a << (32-22));
        maj = (a & b) ^ (a & c) ^ (b & c);
        t2 = S0 + maj;
        S1 = (e >> 6 | e << (32-6)) ^ (e >> 11 | e << (32-11)) ^ (e >> 25 | e << (32-25));
        ch = (e & f) ^ ((~e) & g);
        t1 = h + S1 + ch + sha_k[n] + w[n];

        h = g; g = f; f = e; e = d + t1;
        d = c; c = b; b = a; a = t1 + t2;
        # w[n] = new_var
        w = jax.ops.index_update(w_,jax.ops.index[n-16],new_var)
    return a,b,c,d

In [9]:
import numpy as np

In [10]:

key = jax.random.PRNGKey(0)

def getRandom(shape):
    return jax.random.randint(key = jax.random.PRNGKey(0),minval=0,maxval=1000,shape=shape,dtype=np.uint32)

In [11]:
getRandom((1,))

DeviceArray([302], dtype=uint32)

In [ ]:
jax.make_jaxpr(func)(w)

In [ ]:
w = jax.device_put(w)

In [ ]:
func = jax.jit(func)

In [13]:
batch = 10000
batch = 10
# batch1 = 10
batch2 = 1000
batch2 = 10
# w = getRandom((64-12,batch,batch2))
# w = getRandom((64-12,batch))
w = getRandom((64-12,batch,batch2))
# w = jax.device_put(w)
# %time _ = jax.jit(func)(w)

In [ ]:
func = jax.jit(jax.vmap(func,in_axes=1))

In [ ]:
%time _ = func(w)[0].block_until_ready()

CPU times: user 2.14 ms, sys: 2.05 ms, total: 4.19 ms
Wall time: 8.03 ms


In [ ]:
from jax.experimental.maps import xmap,mesh

In [ ]:
devices = np.array(jax.devices()).reshape((4,2))


In [ ]:
%%time
with mesh(devices, ('x','y')):
    distributed_out = xmap(
        func,
        in_axes=({1: 'left', 2: 'right'}),
        out_axes=['left', 'right', ...],
        axis_resources={'left': 'x', 'right': 'y'})(w)

CPU times: user 33 ms, sys: 59.5 ms, total: 92.6 ms
Wall time: 150 ms


/usr/local/lib/python3.7/dist-packages/jax/experimental/maps.py:418: UserWarning: xmap is an experimental feature and probably has bugs!
  warn("xmap is an experimental feature and probably has bugs!")
